<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2021/aoc2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) number of increments, window length 1 (part 1), window length 3 (part 2)

In [ ]:
wl = 1  # window length, use 3 for part 2
inc = 0 # increments

m = [int(line) for line in open('drive/MyDrive/AoC/2021/input01.txt')]

for i in range(0, len(m) - wl):
  if m[i] < m[i+wl]:
    inc += 1

inc

1722

2) horizontal/depth course

In [ ]:
prog = [line.split() for line in open('drive/MyDrive/AoC/2021/input02.txt')]  # 0=cmd 1=amount
h = 0
d = 0
for instr in prog:
  cmd = instr[0]
  amount = int(instr[1])
  if cmd == 'forward':
    h += amount
  elif cmd == 'up':
    d -= amount
  elif cmd == 'down':
    d += amount

h * d


1698735

In [ ]:
prog = [line.split() for line in open('drive/MyDrive/AoC/2021/input02.txt')]  # 0=cmd 1=amount
h = 0
d = 0
aim = 0 
for instr in prog:
  cmd = instr[0]
  amount = int(instr[1])
  if cmd == 'forward':
    h += amount
    d += aim * amount
  elif cmd == 'up':
    aim -= amount
  elif cmd == 'down':
    aim += amount

h * d


1594785890

3) Most / least common bits in binary numbers represented as equal length strings

In [ ]:
# count values per bit, keep either most common or least common value for each bit
from functools import reduce

ms = [line.strip() for line in open('drive/MyDrive/AoC/2021/input03.txt')]  # ms = measurements, binary 12 char strings
half = len(ms)/2

# count 1s in each bit position
initial_bit_count1s = [0] * len(ms[0])
count1s = reduce(lambda cnt1s, m: [cnt1s[i] + int(m[i]) for i, x in enumerate(cnt1s)], ms, initial_bit_count1s)

most_common_bitvals = "".join(['1' if c > half else '0' for c in count1s])
least_common_bitvals = "".join(['1' if c < half else '0' for c in count1s])  # could also invert most_common_bitvals

int(most_common_bitvals, 2) * int(least_common_bitvals, 2)

4138664

In [ ]:
# count values for first bit, keep lines with first bit either most or least common value,
# repeat with next bit on filtered list, repeat until only one list entry is left
measurements = [line.strip() for line in open('drive/MyDrive/AoC/2021/input03.txt')]  # binary 12 char strings
bits = len(measurements[0])

# find the measurement according to filtermode
# ms - measurements
# filtermode 1 - keep numbers with bit value = most common value of the bit (equally common - keep those with 1)
# filtermode 0 - keep numbers with bit value = least common value of the bit (equally common - keep those with 0)
def find(ms, filtermode):

  def filter(ms, bit):
    half = len(ms)/2
    # count 1s in bit position
    count1s = sum([int(m[bit]) for m in ms])
    # filter according to count1s and filtermode
    filterval = filtermode if count1s > half else (1-filtermode if count1s < half else filtermode)
    return [m for m in ms if int(m[bit]) == filterval]

  def iter(ms, bit):
    filtered = filter(ms, bit)
    return filtered[0] if len(filtered) == 1 else iter(filtered, bit+1)

  return iter(ms, 0)

int(find(measurements, 1), 2) * int(find(measurements, 0), 2)

4273224

4) Bingo (5x5 Boards)


In [ ]:
# Board structure
# 2-dimensional array of pairs (2-elem lists) with left(0) = number, right(1) = marker (1 = marked, 0 = not marked)
# The number/marker pairs are shared between all boards.

from functools import reduce

# Assume numbers drawn between 0 and 99 (incl.), initialize a marker object (a list length 2) for each number,
# containing the number itself (index 0, actually only needed for display as the index of the marker object 
# already represents the number) and a marker (index 1) of value 0 if the number has not been drawn, yet, and
# 1 if the number was drawn. These markers are shared between all the boards, such that setting the marker to 1
# in one of the numbers, the number will appear as marked on all boards containing it.

markers = [[n, 0] for n in range(0, 100)]
boards = []

for i, l in enumerate(open('drive/MyDrive/AoC/2021/input04.txt')):
  if i == 0:
    # The numbers that are drawn appear in the first line of the input
    numbers = [int(s.strip()) for s in l.split(',')]
  elif i % 6 == 1: 
    # A blank line announcing a new 5x5 board, lines 1, 7, 13, etc.
    boards.append([])
  else:
    # By using the numbers on the board def. as index into markers, the marker objects are shared between all the boards
    boards[int((i-1)/6)].append([markers[int(s.strip())] for s in l.split()])

# Some helpers

def mark(n):
  markers[n][1] = 1  # marks number on all boards

def rowbingo(board) -> bool:
  # bool(list) is True if non-empty, i. e. 5 marked numbers in at least 1 row
  return bool([row for row in board if sum([n[1] for n in row]) == 5])  

def colbingo(board) -> bool:
  # The reduce counts marked numbers (row[i][1] == 1) in all columns (i=0..4)
  # true if 5 numbers are marked in any column
  return 5 in reduce(lambda counts, row: [c+row[i][1] for i, c in enumerate(counts)], board, [0, 0, 0, 0, 0])

def bingo(board) -> bool:
  return rowbingo(board) or colbingo(board)

def sum_unmarked(board) -> int:
  return sum([sum([n[0] for n in row if n[1] == 0]) for row in board])

def p(board):
  for row in board:
    print(row)

# Part 1: First board to win
bingoboards = []
index = 0
while len(bingoboards) == 0:
  mark(numbers[index])
  bingoboards = list(filter(bingo, boards))  # [board for board in boards if bingo(board)]
  if bingoboards:
    print("First board")
    p(bingoboards[0])
    print(sum_unmarked(bingoboards[0]) * numbers[index])
  else:
    index += 1

# Part 2: Last board to win

# Now need to remove boards with bingo until only one left
# (first remove the one found above)
boards.remove(bingoboards[0])
while len(boards) > 1:
  # print(f"index: {index}, number of boards: {len(boards)}")
  mark(numbers[index])
  boards = [board for board in boards if not bingo(board)]  # list(filter(lambda board: not(bingo(board)), boards))
  index += 1

print("Last board")
# Go on until bingo
while index < len(numbers):
  mark(numbers[index])
  if (bingo(boards[0])):
    p(boards[0])
    print(sum_unmarked(boards[0]) * numbers[index])
    index = len(numbers)  # stop
  else:
    index += 1



First board
[[6, 0], [26, 0], [69, 1], [27, 1], [75, 0]]
[[61, 0], [33, 1], [88, 1], [38, 0], [20, 0]]
[[9, 1], [56, 0], [70, 0], [98, 1], [82, 0]]
[[80, 1], [76, 0], [55, 1], [66, 1], [29, 0]]
[[97, 1], [84, 1], [42, 1], [77, 1], [73, 1]]
41503
Last board
[[94, 1], [38, 1], [96, 1], [44, 1], [46, 0]]
[[10, 1], [14, 1], [75, 0], [97, 1], [76, 1]]
[[7, 1], [61, 1], [56, 1], [36, 1], [1, 0]]
[[81, 0], [67, 1], [49, 1], [78, 1], [86, 1]]
[[31, 1], [65, 1], [88, 1], [24, 0], [63, 1]]
3178


5) Overlapping lines

In [ ]:
# Part 1 only horizontal and vertical lines
# Part 2 also 45 degree diagonal lines
import re

grid = {}  # maps points (x, y) to count

# needed for part 2
def sgn(x): 
  return 1 if x > 0 else (-1 if x < 0 else 0)

def inc_point(x, y):
  if (x, y) not in grid:
    grid[(x, y)] = 1
  else:
    grid[(x, y)] += 1

def add_line(x1, y1, x2, y2):
  if x1==x2:
    for y in range(min(y1,y2), max(y1,y2)+1):
      inc_point(x1, y)
  elif y1==y2:
    for x in range(min(x1,x2), max(x1,x2)+1):
      inc_point(x, y1)
  # Part 2 - could also use this as single option without all the ifs (trusting the input) as it would cover horizontal and vertical lines, too
  elif abs(x2-x1)==abs(y2-y1):
    dx = sgn(x2-x1)
    dy = sgn(y2-y1)
    for i in range(0, abs(x2-x1)+1):
      inc_point(x1 + i * dx, y1 + i * dy)

for l in open('drive/MyDrive/AoC/2021/input05.txt'):
  m = re.fullmatch(r"(\d+),(\d+) -> (\d+),(\d+)\n", l)
  add_line(*[int(m.group(i)) for i in [1, 2, 3, 4]])

print(f"number of points where lines overlap: {sum([1 if grid[k] > 1 else 0 for k in grid.keys()])}")

number of points where lines overlap: 22083


6) Population growth

In [ ]:
# Part 1 naive brute force
population = [int(timer_str) for timer_str in open('drive/MyDrive/AoC/2021/input06.txt').readline().split(',')]
iterations = 80
for i in range(0, iterations):
  # print(f"{i}: {len(population)}")
  new_timers = [8] * sum([1 if timer == 0 else 0 for timer in population])
  population = [6 if timer == 0 else timer-1 for timer in population]
  population += new_timers

print(f"{iterations}: {len(population)}")


80: 380612


In [ ]:
# Part 2 a bit more intelligent
pop_raw = [int(timer_str) for timer_str in open('drive/MyDrive/AoC/2021/input06.txt').readline().split(',')]
pop_counts = [0,0,0,0,0,0,0,0,0]  # counts for timer values 0-8
for timer in pop_raw:
  pop_counts[timer] += 1
iterations = 256
for i in range(0, iterations):
  # print(f"{i}: {pop_counts} ({sum(pop_counts)})")
  new_counts = pop_counts[1:]  # decrease all from 1-8
  new_counts[6] += pop_counts[0]  # 0s become 6s
  pop_counts = new_counts + [pop_counts[0]]  # 0s produce new 8s

print(f"{iterations}: {pop_counts} ({sum(pop_counts)})")


256: [145229151101, 175734577513, 183417971628, 196504736072, 231271638229, 222902115155, 282121656192, 119339467551, 153645343459] (1710166656900)


7) Find min diff

In [ ]:
positions = [int(s) for s in open('drive/MyDrive/AoC/2021/input07.txt').readline().split(',')]
# input = [16,1,2,0,4,2,7,1,2,14]
print(f"estimation(part 1): {sum(positions)/len(positions)/2}")

def consumption1(target, current):
  return sum([abs(target - curr) for curr in current])

def consumption2(target, current):
  return sum([int(abs(target-curr)*(abs(target-curr)+1)/2) for curr in current])

print(min([(i, consumption1(i, positions)) for i in range(0, len(positions))], key=lambda t: t[1]))
print(min([(i, consumption2(i, positions)) for i in range(0, len(positions))], key=lambda t: t[1]))
  

estimation(part 1): 243.2525
(347, 347449)
(486, 98039527)
